**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [23]:
!pip install scikit-video

In [24]:
!sudo apt install skvideo

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package skvideo


In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.regularizers import l2
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten
np.random.seed(1)



# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass
    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
<font color='blue'>__Question 1__:</font>
Explain the function act. Why is ```epsilon``` essential?

1) The act function , given the current state of the game and the type of phase either training or testing, returns the action to take at the next step.
* For the training phase : we use an $\epsilon$ greedy algorithm that return a random action with a probability of $\epsilon$ and return an action based on the learnt policy with a pobability of $1-\epsilon$.
* For the testing phase : the agent takes the best action he learnt from the current state.

2) The ```epsilon``` importance resides in its role to push the agent to explore the whole environment and prevents from being stuck in local maximas for the reward.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test= 15 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

<font color='blue'>__Question 2__</font> Explain the use of the arrays ```position``` and ```board```.

- ```position``` is an NxN matrix, used to represent the position of the rat at step t. The position of the rat is modelled by the value 1. The value 0 corresponds to alloweed states and the value 1 corresponds to value the forbidden states (borders of the grid)

- ```board``` is a NxN matrix taht reprensets the reward for each cell of the grid: -1 for poisonous cells and 0.5 for positive cells.


## Random Agent

***
<font color='blue'> __Question 3__</font> Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
<font color='blue'>__Question 4__</font> Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
    
        state = env.reset()
        # Initialization at each epoch
        game_over = False
        win = 0
        lose = 0
        
        while not game_over:
            
            #Agent takes an action
            action=agent.act(state)
            #Update: next state, reward and a boolean variable indicating the end(or not) of the game from the environment
            prev_state = state
            state, reward, game_over = env.act(action)
 
            
            #Update the score
            if reward>0:
                win = win + reward
            if reward<0:
                lose = lose - reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [36]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.0/12.0. Average score (-1.0)
Win/lose count 11.0/12.0. Average score (-1.0)
Win/lose count 11.0/15.0. Average score (-2.0)
Win/lose count 14.0/11.0. Average score (-0.75)
Win/lose count 7.5/13.0. Average score (-1.7)
Win/lose count 7.0/19.0. Average score (-3.4166666666666665)
Win/lose count 11.5/18.0. Average score (-3.857142857142857)
Win/lose count 14.5/14.0. Average score (-3.3125)
Win/lose count 9.5/7.0. Average score (-2.6666666666666665)
Win/lose count 15.5/18.0. Average score (-2.65)
Win/lose count 12.0/15.0. Average score (-2.6818181818181817)
Win/lose count 8.5/14.0. Average score (-2.9166666666666665)
Win/lose count 13.5/19.0. Average score (-3.1153846153846154)
Win/lose count 10.5/8.0. Average score (-2.7142857142857144)
Win/lose count 12.0/10.0. Average score (-2.4)
Final score: -2.4


***
## DQN

Let us assume here that $T=\infty$.

***
<font color='blue'>__Question 5__ </font>Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**1) Q function expression**

$$ Q^\pi(s,a) = E_{p^\pi}[\sum_{t=0}^\infty \gamma^t r_t(s_t,a_t)| s_0 =s, a_0=a] \\ = r(s,a) + E_{p^\pi}[\sum_{t=1}^\infty \gamma^t r(s_t,a_t)| s_0 =s, a_0=a]   $$
                
With a variable change $k=t-1$ :

$ Q^\pi(s,a) = r(s,a) + \gamma E_{p^\pi}[\sum_{k=0}^\infty \gamma^k r(s_k,a_k)| s_0 =s, a_0=a] \\= r(s,a) + \gamma E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[\sum_{k=0}^\infty \gamma^t r(s_k,a_k)| s_0 =s', a_0=a']]\\ = E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[ r(s,a) + \sum_{k=0}^\infty \gamma^k r(s_k,a_k)| s_0 =s', a_0=a']] \\=  E_{(s',a')\sim p(.|s,a)}[r(s,a) + \gamma  Q^\pi(s',a') ]$


**2) Optimal Q function expression**

We know that the optimal policy (if it exists) $\pi^*$ is : $\pi^*(s') = argmax_{a'} Q^*(s',a') $


and we know that :

$ Q^*(s,a) = E_{(s',a')\sim \pi^*(.|s,a)}[r(s,a) + \gamma max_\pi Q^\pi(s',a')] \\= E_{(s',a')\sim p(.|s,a)}[ r(s,a) + \gamma Q^*(s',a')]\\= E_{(s'\sim p^*(.|s,a),a'\sim \pi^*(.|s'))}[r(s,a) + \gamma Q^*(s',a')] $

since $\pi^* $ is deterministic, then:

$ Q^*(s,a) = E_{(s'\sim p^*(.|s,a),a'=\pi^*(s'))}[r(s,a) + \gamma Q^*(s',a')] \\ =  E_{s'\sim \pi^*(.|s,a)}[r(s,a) + \gamma max_{a'}Q^*(s',a')]  $



**3) Loss function expression**

The ultimate goal is to approximate $Q^*(s,a)$ , therefore we can define the following loss function :

$$ \mathcal{L}(\theta) = E_{s'\sim \pi^*(.|s,a)}[||Q^*(s,a) - Q(s,a,\theta)||^2] $$

By using the result of the previous question $$Q^*(s,a) = E_{s'\sim \pi^*(.|s,a)}[r(s,a) + \gamma max_{a'}Q^*(s',a')] $$

The final loss is therefore:

$$ \mathcal{L}(\theta) = E_{s'\sim \pi^*(.|s,a)}[|| r(s,a) + \gamma max_{a'}Q^*(s',a') - Q(s,a,\theta) ||^2]$$




***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
<font color='blue'>__Question 6__ </font>Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action

            #Agent takes an action
            action=agent.act(state)
            
            #Update: next state, reward and a boolean variable indicating the end(or not) of the game from the environment
            prev_state = state
            state, reward, game_over = env.act(action)
 
            

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        # Save as a mp4
        if (e) % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
<font color='blue'>__Question 7__</font> Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # Update from environment
            s_, n_s_, a_, r_, game_over_=self.memory.random_access()
            # Store current values
            states[i] = s_
            target_q[i] = self.model.predict(s_[None])
            
            #  Update Q_star
            if game_over_:
                target_q[i,a_] = r_
               
            else:
                #q_hat = self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
                q_hat = self.model.predict(n_s_[None])
                target_q[i,a_] = r_+ self.discount* np.max(q_hat,axis=1)

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -5, 5)

        l = self.model.train_on_batch(states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state, )))
        model.add(Dense(32, input_shape=(5*5*self.n_state, ), activation= 'relu', kernel_regularizer = l2(1e-4)))
        model.add(Dense(4,input_shape=(5*5*self.n_state, ), activation= 'relu', kernel_regularizer = l2(1e-4)))      
        
        model.compile(sgd(lr=lr, decay=1e-4,momentum=0.0), "mse")
        self.model = model
        

In [43]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0126 | Win/lose count 7.0/4.0 (3.0)
Epoch 001/050 | Loss 0.0070 | Win/lose count 1.0/2.0 (-1.0)
Epoch 002/050 | Loss 0.0112 | Win/lose count 6.0/3.0 (3.0)
Epoch 003/050 | Loss 0.0177 | Win/lose count 3.5/8.0 (-4.5)
Epoch 004/050 | Loss 0.0103 | Win/lose count 2.0/4.0 (-2.0)
Epoch 005/050 | Loss 0.0209 | Win/lose count 5.5/1.0 (4.5)
Epoch 006/050 | Loss 0.2079 | Win/lose count 8.0/4.0 (4.0)
Epoch 007/050 | Loss 0.0220 | Win/lose count 2.5/2.0 (0.5)
Epoch 008/050 | Loss 0.0101 | Win/lose count 1.0/0 (1.0)
Epoch 009/050 | Loss 0.1922 | Win/lose count 3.5/3.0 (0.5)
Epoch 010/050 | Loss 0.2011 | Win/lose count 5.5/2.0 (3.5)
Epoch 011/050 | Loss 0.0087 | Win/lose count 3.0/2.0 (1.0)
Epoch 012/050 | Loss 0.0108 | Win/lose count 3.0/3.0 (0.0)
Epoch 013/050 | Loss 0.1924 | Win/lose count 6.0/2.0 (4.0)
Epoch 014/050 | Loss 0.0124 | Win/lose count 3.5/2.0 (1.5)
Epoch 015/050 | Loss 0.0128 | Win/lose count 3.0/1.0 (2.0)
Epoch 016/050 | Loss 0.0067 | Win/lose count 3.0/3.0 (0

<font color='green'> **Comments** </font> Finetuning on:

- For Learning Rate :  lr = 0.1, lr = 0.2 and lr=0.05
- For batch size 16 , 32, 64 
- Memory size 2000, 5000
- Optimizer: Sgd or Adam

--> The best values are: **LR=0.1, batch_size=32,Memory_size=2000, Sgd**


***
***
<font color='blue'>__Question 8__</font> Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model=Sequential()
        model.add(Conv2D(32, kernel_size=3, input_shape=(5,5,self.n_state,),activation='relu',kernel_regularizer = l2(1e-4)))
        model.add(Conv2D(64, kernel_size=3, activation='relu',kernel_regularizer = l2(1e-4)))
        model.add(Flatten())
        model.add(Dense(units = 4,kernel_regularizer = l2(1e-4)))
        
        model.compile(sgd(lr=lr, decay=1e-4,momentum=0.0), "mse")
        self.model = model

In [45]:

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0065 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/050 | Loss 0.0108 | Win/lose count 4.0/3.0 (1.0)
Epoch 002/050 | Loss 0.0103 | Win/lose count 3.0/5.0 (-2.0)
Epoch 003/050 | Loss 0.0131 | Win/lose count 1.0/0 (1.0)
Epoch 004/050 | Loss 0.0072 | Win/lose count 3.5/4.0 (-0.5)
Epoch 005/050 | Loss 0.0114 | Win/lose count 3.0/3.0 (0.0)
Epoch 006/050 | Loss 0.0098 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/050 | Loss 0.0127 | Win/lose count 4.5/5.0 (-0.5)
Epoch 008/050 | Loss 0.0080 | Win/lose count 4.5/3.0 (1.5)
Epoch 009/050 | Loss 0.0071 | Win/lose count 2.5/2.0 (0.5)
Epoch 010/050 | Loss 0.0088 | Win/lose count 6.5/3.0 (3.5)
Epoch 011/050 | Loss 0.3398 | Win/lose count 5.5/1.0 (4.5)
Epoch 012/050 | Loss 0.0099 | Win/lose count 4.5/4.0 (0.5)
Epoch 013/050 | Loss 0.0094 | Win/lose count 5.0/2.0 (3.0)
Epoch 014/050 | Loss 0.0071 | Win/lose count 6.0/3.0 (3.0)
Epoch 015/050 | Loss 0.0091 | Win/lose count 10.0/8.0 (2.0)
Epoch 016/050 | Loss 0.1608 | Win/lose count 8.0/3.0 

***
***
<font color='blue'>__Question 9__</font> Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [48]:

env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')
epochs_test = 10
agent_fc = DQN_FC(size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 132)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 5.0/2.0. Average score (3.0)
Win/lose count 2.5/0. Average score (2.75)
Win/lose count 6.5/2.0. Average score (3.3333333333333335)
Win/lose count 5.5/2.0. Average score (3.375)
Win/lose count 6.0/4.0. Average score (3.1)
Win/lose count 8.5/6.0. Average score (3.0)
Win/lose count 9.0/4.0. Average score (3.2857142857142856)
Win/lose count 1.0/2.0. Average score (2.75)
Win/lose count 8.5/3.0. Average score (3.0555555555555554)
Win/lose count 8.0/1.0. Average score (3.45)
Final score: 3.45
Test of the FC
Win/lose count 4.5/4.0. Average score (0.5)
Win/lose count 6.0/1.0. Average score (2.75)
Win/lose count 5.5/15.0. Average score (-1.3333333333333333)
Win/lose count 6.5/2.0. Average score (0.125)
Win/lose count 4.5/3.0. Average score (0.4)
Win/lose count 5.5/5.0. Average score (0.4166666666666667)
Win/lose count 6.5/6.0. Average score (0.42857142857142855)
Win/lose count 2.5/4.0. Average score (0.1875)
Win/lose count 3.5/2.0. Average score (0.3333333333333333

In [49]:
HTML(display_videos('cnn_test9.mp4'))

In [51]:
HTML(display_videos('fc_test9.mp4'))

<font color='green'> **Comments:** </font>

1) **The issues:** one main issue is that the agent does not explore the whole map, as it seems to oscillate between two positions whenever there is no positive rewards during his exploration.

2) **Increasing temperature:** Increasing the temperture results in an increase in the number of positive rewards, therefore the agent will explore more areas because the positive rewards are nearby. As a sresult, the overall score will be higher.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
<font color='blue'>__Question 10__</font> Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,epsilon,decay=0.9,prefix=''):
    score = 0
    loss = 0
    
    for e in range(epoch):
        # initialization
        state = env.reset()

        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(decay * agent.epsilon)

        while not game_over:
            #Agent takes an action
            action=agent.act(state)
            
            #Update: next state, reward from the environment
            prev_state = state
            state, reward, game_over = env.act(action,train=True)
 
            # Update the counters
            if reward > 0:
                win +=  reward
            if reward < 0:
                lose -= reward
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        

        # Save as a mp4
        if (e) % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.3f}/{:.3f} ({:.3f}), epsilon {:.3f}"
              .format(e, epoch, loss, win, lose, win-lose, epsilon))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
                
            


class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
    
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b

    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        reward = self.board[self.x, self.y] - train * self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))

        malus[bonus>0]=0
        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
        
    

In [54]:
# Training
# NEw parameters 
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1,epsilon = 0.9, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, epsilon = 0.9,decay =0.8,prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0276 | Win/lose count 9.500/23.100 (-13.600), epsilon 0.900
Epoch 001/050 | Loss 0.0146 | Win/lose count 11.500/22.200 (-10.700), epsilon 0.900
Epoch 002/050 | Loss 0.0142 | Win/lose count 5.000/22.700 (-17.700), epsilon 0.900
Epoch 003/050 | Loss 0.0154 | Win/lose count 9.500/21.300 (-11.800), epsilon 0.900
Epoch 004/050 | Loss 0.0144 | Win/lose count 9.000/17.800 (-8.800), epsilon 0.900
Epoch 005/050 | Loss 0.0146 | Win/lose count 16.000/17.700 (-1.700), epsilon 0.900
Epoch 006/050 | Loss 0.0122 | Win/lose count 17.000/17.900 (-0.900), epsilon 0.900
Epoch 007/050 | Loss 0.0157 | Win/lose count 18.500/16.000 (2.500), epsilon 0.900
Epoch 008/050 | Loss 0.0161 | Win/lose count 21.000/12.600 (8.400), epsilon 0.900
Epoch 009/050 | Loss 0.0153 | Win/lose count 5.000/17.200 (-12.200), epsilon 0.900
Epoch 010/050 | Loss 0.0165 | Win/lose count 12.000/16.000 (-4.000), epsilon 0.900
Epoch 011/050 | Loss 0.0155 | Win/lose count 19.500/13.400 (6.100), epsilon 0.900
Epoch 0

In [55]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 10.5/0. Average score (10.5)
Win/lose count 6.0/0. Average score (8.25)
Win/lose count 4.0/0. Average score (6.833333333333333)
Win/lose count 3.5/0. Average score (6.0)
Win/lose count 9.0/1.0. Average score (6.4)
Win/lose count 12.0/1.0. Average score (7.166666666666667)
Win/lose count 2.0/0. Average score (6.428571428571429)
Win/lose count 9.0/0. Average score (6.75)
Win/lose count 4.5/0. Average score (6.5)
Win/lose count 7.0/0. Average score (6.55)
Final score: 6.55


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***